In [2]:
import requests
import pandas as pd
from datetime import datetime
import os

In [13]:
# Função para buscar dados da API do Banco Central
def fetch_bcb_series(series_code, start_date="20000101", end_date=None):
    """
    Busca uma série do Banco Central do Brasil usando a API SGS.
    
    Args:
        series_code (int): Código da série.
        start_date (str): Data de início no formato AAAAMMDD (default: 2000-01-01).
        end_date (str): Data de fim no formato AAAAMMDD (default: hoje).
    
    Returns:
        pd.DataFrame: DataFrame com as datas e valores da série.
    """
    if end_date is None:
        end_date = datetime.today().strftime('%d/%m/%Y')
    
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_code}/dados"
    params = {"formato": "json", "dataInicial": start_date, "dataFinal": end_date}
    print(url)
    print(params)
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    data = response.json()
    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
    return df

In [25]:
# Função para buscar dados da API do IBGE (SIDRA)
def fetch_ibge_series(table_id, variable_id, period="all"):
    """
    Busca dados do IBGE usando a API SIDRA.
    
    Args:
        table_id (int): Código da tabela no SIDRA.
        variable_id (int): Código da variável na tabela.
        period (str): Período de busca (default: "all").
    
    Returns:
        pd.DataFrame: DataFrame com os dados da série.
    """
    url = f"https://apisidra.ibge.gov.br/values/t/{table_id}/v/{variable_id}/p/{period}/N1/1"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    print(url)
    
    # Converter JSON para DataFrame
    df = pd.DataFrame(data[1:])  # Ignora o cabeçalho
    df['data'] = pd.to_datetime(df['D2C'].str[:4] + '-' + (df['D2C'].str[4:].astype(int) * 3 - 2).astype(str), format='%Y-%m')
    df['valor'] = pd.to_numeric(df['V'], errors='coerce')
    return df[['data', 'valor']]

In [30]:
def fetch_pib_data():
    """
    Busca os dados do PIB trimestral usando a API do IBGE.
    """
    url = "https://servicodados.ibge.gov.br/api/v3/agregados/6613/periodos/199601|199602|199603|199604|199701|199702|199703|199704|199801|199802|199803|199804|199901|199902|199903|199904|200001|200002|200003|200004|200101|200102|200103|200104|200201|200202|200203|200204|200301|200302|200303|200304|200401|200402|200403|200404|200501|200502|200503|200504|200601|200602|200603|200604|200701|200702|200703|200704|200801|200802|200803|200804|200901|200902|200903|200904|201001|201002|201003|201004|201101|201102|201103|201104|201201|201202|201203|201204|201301|201302|201303|201304|201401|201402|201403|201404|201501|201502|201503|201504|201601|201602|201603|201604|201701|201702|201703|201704|201801|201802|201803|201804|201901|201902|201903|201904|202001|202002|202003|202004|202101|202102|202103|202104|202201|202202|202203|202204|202301|202302|202303|202304|202401|202402/variaveis/9319?localidades=N1[all]&classificacao=11255[90707]"
    
    response = requests.get(url)
    response.raise_for_status()
    
    data = response.json()
    
    # Estruturar os dados no DataFrame
    results = []
    for entry in data[0]['resultados']:
        for serie in entry['series']:
            for value in serie['serie'].items():
                period, pib_value = value
                results.append({'periodo': period, 'pib': float(pib_value)})
    
    df = pd.DataFrame(results)
    
    # Converter o período para formato datetime
    df['data'] = pd.to_datetime(df['periodo'].str[:4] + '-' + (df['periodo'].str[4:].astype(int) * 3 - 2).astype(str), format='%Y-%m')
    df = df.drop(columns=['periodo'])  # Opcional: remover coluna original
    return df

In [ ]:
# Lista de séries do BCB e seus códigos
bcb_series = {
    "Selic_Meta": 432,
    "Taxa_Juros_PF": 20786,
    "Saldo_Credito_PF": 20634,
    "Inadimplencia_PF": 21084,
    "Captacao_Liquida_Poupanca": 240,
    "Saldo_Poupanca": 7836,
    "Confiança_Consumidor": 4393,
    "IPCA": 433
}

# Data inicial para busca
start_date = "01/01/2000"

# Loop para buscar todas as séries do BCB
for name, code in bcb_series.items():
    print(f"Baixando série do BCB: {name} (Código: {code})")
    df = fetch_bcb_series(code, start_date)
    df = df.rename(columns={"valor": name})
    filename = f"bcb {name} {code} {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"
    df.to_csv(filename, index=False, sep=";")
    print(f"Arquivo salvo: {filename}")


Baixando série do BCB: Selic_Meta (Código: 432)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '11/11/2024'}
Arquivo salvo: bcb Selic_Meta 432 20241111143730.csv
Baixando série do BCB: Taxa_Juros_PF (Código: 20786)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.20786/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '11/11/2024'}
Arquivo salvo: bcb Taxa_Juros_PF 20786 20241111143730.csv
Baixando série do BCB: Saldo_Credito_PF (Código: 20634)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.20634/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '11/11/2024'}
Arquivo salvo: bcb Saldo_Credito_PF 20634 20241111143730.csv
Baixando série do BCB: Inadimplencia_PF (Código: 21084)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.21084/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '11/11/2024'}
Arquivo salvo: bcb Inadimplencia_PF 21084 20241111143731.csv
Baixando série do BC

In [28]:
#/T/  Tabela:  	6613 - 	Valores encadeados a preços de 1995 com ajuste sazonal
#9319  Valores encadeados a preços de 1995 com ajuste sazonal (Milhões de Reais) - casas decimais: padrão = 2, máximo = 4
# 90707  PIB a preços de mercado

def fetch_pib_data():
    """
    Busca os dados do PIB trimestral usando a API do IBGE.
    """
    url = "https://servicodados.ibge.gov.br/api/v3/agregados/6613/periodos/199601|199602|199603|199604|199701|199702|199703|199704|199801|199802|199803|199804|199901|199902|199903|199904|200001|200002|200003|200004|200101|200102|200103|200104|200201|200202|200203|200204|200301|200302|200303|200304|200401|200402|200403|200404|200501|200502|200503|200504|200601|200602|200603|200604|200701|200702|200703|200704|200801|200802|200803|200804|200901|200902|200903|200904|201001|201002|201003|201004|201101|201102|201103|201104|201201|201202|201203|201204|201301|201302|201303|201304|201401|201402|201403|201404|201501|201502|201503|201504|201601|201602|201603|201604|201701|201702|201703|201704|201801|201802|201803|201804|201901|201902|201903|201904|202001|202002|202003|202004|202101|202102|202103|202104|202201|202202|202203|202204|202301|202302|202303|202304|202401|202402/variaveis/9319?localidades=N1[all]&classificacao=11255[90707]"
    
    response = requests.get(url)
    response.raise_for_status()
    
    data = response.json()
    
    # Estruturar os dados no DataFrame
    results = []
    for entry in data[0]['resultados']:
        for serie in entry['series']:
            for value in serie['serie'].items():
                period, pib_value = value
                results.append({'periodo': period, 'pib': float(pib_value)})
    
    df = pd.DataFrame(results)
    
    # Converter o período para formato datetime
    df['data'] = pd.to_datetime(df['periodo'], format='%Y%m')
    df = df.drop(columns=['periodo'])  # Opcional: remover coluna original
    return df

In [ ]:
#/T/  Tabela:  	1620 - 	Série encadeada do índice de volume trimestral (Base: média 1995 = 100)
#583  Série encadeada do índice de volume trimestral (Base: média 1995 = 100) (Número-índice) - casas decimais: padrão = 2, máximo = 4

#/T/  Tabela:  	4094 - 	Pessoas de 14 anos ou mais de idade, total, na força de trabalho, ocupadas, desocupadas, fora da força de trabalho, em situação de informalidade e respectivas taxas e níveis, por grupo de idade
# 4110  Distribuição percentual das pessoas de 14 anos ou mais de idade, desocupadas na semana de referência (%) - casas decimais: padrão = 1, máximo = 1

# Lista de séries do IBGE e suas tabelas/variáveis
ibge_series = {
    "PIB_Trimestral": {"table_id": 1620, "variable_id": 583},  # PIB mensal (exemplo)
    "Indice_Desemprego": {"table_id": 4094, "variable_id": 4110},  # Taxa de desemprego
}

# Loop para buscar todas as séries do IBGE
for name, params in ibge_series.items():
    print(f"Baixando série do IBGE: {name} (Tabela: {params['table_id']}, Variável: {params['variable_id']})")
    df = fetch_ibge_series(params['table_id'], params['variable_id'])
    df = df.rename(columns={"valor": name})
    filename = f"ibge {name} {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"
    df.to_csv(filename, index=False, sep=";")
    print(f"Arquivo salvo: {filename}")
    


Baixando série do IBGE: PIB_Mensal (Tabela: 1620, Variável: 583)
https://apisidra.ibge.gov.br/values/t/1620/v/583/p/all/N1/1
Arquivo salvo: ibge PIB_Mensal 20241111145146.csv
Baixando série do IBGE: Indice_Desemprego (Tabela: 4094, Variável: 4110)
https://apisidra.ibge.gov.br/values/t/4094/v/4110/p/all/N1/1
Arquivo salvo: ibge Indice_Desemprego 20241111145148.csv


In [31]:
# Testar a função
df_pib = fetch_pib_data()
name = 'PIB Desazonalizado Encadeado 1995'
filename = f"ibge {name} {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"

# Exportar para CSV (opcional)
df_pib.to_csv(filename, index=False, sep=";")

In [3]:
def merge_csv_files_in_folder(folder_path):
    """
    Escaneia todos os arquivos .csv em uma pasta e faz o merge usando a coluna 'data' como índice.

    Args:
        folder_path (str): Caminho da pasta onde os arquivos CSV estão armazenados.

    Returns:
        pd.DataFrame: DataFrame combinado.
    """
    merged_df = None
    
    # Escanear arquivos na pasta
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Lendo arquivo: {file_name}")
            # Carregar o arquivo CSV
            df = pd.read_csv(file_path, sep=';', parse_dates=['data'], index_col='data')
            
            # Merge com o DataFrame principal
            if merged_df is None:
                merged_df = df
            else:
                merged_df = merged_df.merge(df, how='outer', left_index=True, right_index=True)
    
    return merged_df

In [4]:
# Caminho da pasta onde os arquivos CSV estão armazenados
folder_path = "./"  # Atualize se necessário

# Chama a função para combinar os arquivos e descartar linhas com dados faltando
merged_df = merge_csv_files_in_folder(folder_path).dropna()

# Exportar o DataFrame combinado para um arquivo CSV (opcional)
merged_df.to_csv("merged_data.csv", sep=';')

Lendo arquivo: bcb Captacao_Liquida_Poupanca 240 20241111143737.csv
Lendo arquivo: bcb Confiança_Consumidor 4393 20241111143738.csv
Lendo arquivo: bcb Inadimplencia_PF 21084 20241111143731.csv
Lendo arquivo: bcb IPCA 433 20241111143739.csv
Lendo arquivo: bcb Saldo_Credito_PF 20634 20241111143730.csv
Lendo arquivo: bcb Saldo_Poupanca 7836 20241111143738.csv
Lendo arquivo: bcb Selic_Meta 432 20241111143730.csv
Lendo arquivo: bcb Taxa_Juros_PF 20786 20241111143730.csv
Lendo arquivo: ibge PIB Desazonalizado Encadeado 1995 20241111150336.csv


In [11]:
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio

# Carregar os dados
file_path = "merged_data.csv"
merged_data = pd.read_csv(file_path, sep=';', parse_dates=['data'], index_col='data')

# Renomear colunas para nomes mais descritivos
renamed_columns = {
    "Taxa_Juros_PF": "Taxa Juros (PF)",
    "Saldo_Credito_PF": "Crédito PF",
    "Inadimplencia_PF": "Inadimplência PF",
    "Captacao_Liquida_Poupanca": "Captação Poupança",
    "Saldo_Poupanca": "Saldo Poupança",
    "Selic_Meta": "Taxa Selic",
    "Confiança_Consumidor": "Confiança Consumidor",
    "IPCA": "Inflação (IPCA)",
    "pib": "PIB"
}
merged_data.rename(columns=renamed_columns, inplace=True)

# Gerar matriz de correlação
correlation_matrix = merged_data.corr().round(2)

# Criar heatmap da matriz de correlação com valores numéricos
fig_corr = go.Figure(
    data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale="Viridis",
        zmin=-1,
        zmax=1,
        colorbar=dict(title="Correlation"),
        text=correlation_matrix.values,  # Adicionar os valores numéricos
        hoverinfo="text"  # Exibir valores ao passar o mouse
    )
)

# Adicionar os números sobre as células
for i, row in enumerate(correlation_matrix.values):
    for j, value in enumerate(row):
        fig_corr.add_trace(
            go.Scatter(
                x=[correlation_matrix.columns[j]],
                y=[correlation_matrix.index[i]],
                text=[f"{value:.2f}"],  # Formatar com duas casas decimais
                mode="text",
                showlegend=False,
                textfont=dict(color="white" if abs(value) > 0.5 else "black")
            )
        )

# Configurar layout do heatmap
fig_corr.update_layout(
    title="Matriz de Correlação - Variáveis Renomeadas",
    xaxis_title="Variáveis",
    yaxis_title="Variáveis",
    xaxis=dict(tickangle=-45),
    hovermode="closest"
)

# Salvar matriz de correlação como HTML
output_corr_file = "correlation_matrix_with_numbers.html"
pio.write_html(fig_corr, output_corr_file)

# Confirmar os arquivos gerados
print(f"Matriz de correlação gerada: {output_corr_file}")


Matriz de correlação gerada: correlation_matrix_with_numbers.html


In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import plotly.io as pio

# Caminho do arquivo CSV
file_path = "merged_data.csv"

# Carregar os dados do arquivo
merged_data = pd.read_csv(file_path, sep=';', parse_dates=['data'], index_col='data')

# Remover colunas ou linhas com valores ausentes, se necessário
merged_data = merged_data.dropna()

# Normalizar os dados
scaler = MinMaxScaler()
normalized_data = pd.DataFrame(
    scaler.fit_transform(merged_data),
    index=merged_data.index,
    columns=merged_data.columns
)

# Criar o gráfico interativo com Plotly
fig = go.Figure()

# Adicionar todas as séries como curvas individuais
for column in normalized_data.columns:
    fig.add_trace(
        go.Scatter(
            x=normalized_data.index,
            y=normalized_data[column],
            mode='lines',
            name=column
        )
    )

# Adicionar botões para manipular a visibilidade
button_all = dict(
    label="Mostrar Todas",
    method="update",
    args=[{"visible": [True] * len(normalized_data.columns)},  # Mostrar todas as curvas
          {"title": "Dados Normalizados - Todas as Curvas Visíveis"}]
)

button_none = dict(
    label="Ocultar Todas",
    method="update",
    args=[{"visible": ["legendonly"] * len(normalized_data.columns)},  # Ocultar todas, mas manter na legenda
          {"title": "Dados Normalizados - Nenhuma Curva Visível"}]
)

# Atualizar o layout para incluir os botões
fig.update_layout(
    title="Dados Normalizados - Adicione ou Remova Curvas",
    xaxis_title="Data",
    yaxis_title="Valores Normalizados",
    legend_title="Variáveis",
    hovermode="x unified",
    updatemenus=[dict(
        type="buttons",
        direction="left",
        buttons=[button_all, button_none],
        showactive=True,
        x=0.1,
        y=1.1
    )]
)

# Salvar o gráfico como HTML
output_file = "normalized_data_plot_with_buttons.html"
pio.write_html(fig, output_file)

# Exibir confirmação
print(f"Gráfico interativo gerado e salvo como: {output_file}")


Gráfico interativo gerado e salvo como: normalized_data_plot_with_buttons.html
